# 5-quasi-regular graph using the OR-tools with CP-SAT solver in python in thhe case of  n=21 and diameter 2
                                                                                  Hailemariam Abebe Tekile
                                                                                     17 March 2023

Reference: 
Szádoczki, Z., Bozóki, S., & Tekile, H. A. (2022). Filling in pattern designs for incomplete pairwise comparison matrices:(quasi-) regular graphs with minimal diameter. Omega, 107, 102557.

The integer Linear programmming problem is the following:
Let $N=\{1,\ldots,21\}$ be the set of vertices, and let $P=\{i \in N,j \in N:i<j\}$ be the pairs of vertices. For $(i,j) \in P$, let $X_{i,j}$ be a binary decision variable that indicates whether $(i,j)$ is an edge. For $(i,j) \in P$ and $k \in N \setminus \{i,j\}$, let $Y_{i,j,k}$ be a binary decision variable that indicates whether $K$ is a common neighbor of $i$ and $j$. For $(i,j) \in P$, let $SLACK_{i,j}$ be a slack variable.
\begin{align}
\min{\sum_{(i,j) \in P}{SLACK_{i,j}}} \hspace{3cm} \label{eq:int0}\\
\sum_{(i,j) \in P: k \in \{i,j\}}{X_{i,j} =5} \hspace{2cm} \text{ for } k \in N\setminus {21} \label{eq:int1}\\
\sum_{(i,j) \in P: k \in \{i,j\}}{X_{i,j} =6} \hspace{2cm} \text{ for } k \in  \{21\}\label{eq:intt1}\\
X_{i,j}+\sum_{k \in N \setminus \{i,j\}}{Y_{i,j,k}} + SLACK_{i,j} \geq 1 \hspace{1cm} \text{ for } (i,j) \in P \label{eq:int2}\\
 Y_{i,j,k} \leq X_{i,k}[\text{if $i<k$}]+X_{k,i}[\text{if $k<i$}]  \text{ for $(i,j)$} \in P \text{ and }k \in N \setminus \{i,j\} \label{eq:int3}\\
 Y_{i,j,k} \leq X_{j,k}[\text{if $j<k$}]+X_{k,j}[\text{if $k<j$}]  \text{ for $(i,j)$} \in P \text{ and }k \in N \setminus \{i,j\} \label{eq:int4}
\end{align}

Note that we use the CP-SAT solver from the OR-Tools library and create Boolean variables using NewBoolVar and integer variables using NewIntVar. We also use Add instead of addConstrs to add constraints to the model. Finally, we use CpSolver instead of Model to solve the model and extract the solution using Value instead of X.

The code uses the CP-SAT solver from the OR-Tools library to solve a graph theory problem. The problem is to find a minimum weight spanning tree in an undirected graph with 21 nodes, where the degree of each node is 5 except for the last node, which has a degree of 6.

The code first defines the number of nodes and their degree using a dictionary called degree, where the key is the node number and the value is its degree.

The code then creates a CP-SAT model object called model and defines three dictionaries X, Y, and Slack.

The X dictionary represents the edges of the graph and is defined using two nested loops that iterate over all pairs of nodes in the graph. For each pair of nodes (i, j), the code creates a new Boolean variable X[(i, j)] and adds it to the model.

The Y dictionary is used to represent auxiliary variables that help to ensure that the selected edges form a tree. For each pair of nodes (i, j) and each node k that is not i or j, the code creates a new Boolean variable Y[(i, j, k)] and adds it to the model.

The Slack dictionary is used to represent a relaxation of the problem constraints. For each pair of nodes (i, j), the code creates a new integer variable Slack[(i, j)] with a domain of [0, 1] and adds it to the model.

The code then defines the objective function to minimize the sum of all Slack variables.

The next set of constraints ensure that each node in the graph has the correct degree. For each node k in the graph, the code adds a new constraint to the model that specifies that the sum of all X variables that involve k must equal its degree.

The next set of constraints ensure that the selected edges form a tree. For each pair of nodes (i, j), the code adds a new constraint to the model that specifies that at least one of X[(i, j)], Y[(i, j, k)], or Slack[(i, j)] must be true.

The final set of constraints ensure that the selected edges do not form cycles. For each pair of nodes (i, j) and each node k that is not i or j, the code adds two new constraints to the model. The first constraint specifies that if Y[(i, j, k)] is true, then either X[(i, k)] or X[(k, i)] must also be true, and the second constraint specifies that if Y[(i, j, k)] is true, then either X[(j, k)] or X[(k, j)] must also be true.

The code then creates a CP-SAT solver object called solver, sets a solver parameter to disable logging of the search progress, and calls the Solve method of the solver object to solve the model.

If the solver returns an optimal solution, the code extracts the values of the X and Y variables and creates a set of edges EDGES that correspond to the selected edges in the graph. Finally, the code prints the values of X, Y, and EDGES.

In [1]:
#Import the cp_model module from the ortools.sat.python package
from ortools.sat.python import cp_model

#Set the number of nodes to 21, and create a list of node labels
n = 21;
NODES = range(1, n+1);
#Create a dictionary that maps each node label to its degree
degree = {k: 5 if k in range(1, n) else 6 for k in NODES};
#Create a new CpModel object
model = cp_model.CpModel()
#Create a dictionary of Boolean variables X that represent the edges of the graph
X = {}
for i in NODES:
    for j in NODES:
        if i < j:
            X[(i, j)] = model.NewBoolVar('X_{}_{}'.format(i, j))
#Create a dictionary of Boolean variables Y that will be used to enforce that at most one edge is selected
#for each triple (i, j, k), where i < j < k and k is not equal to i or j            
Y = {}
for (i, j) in X.keys():
    for k in NODES:
        if k not in (i, j):
            Y[(i, j, k)] = model.NewBoolVar('Y_{}_{}_{}'.format(i, j, k))
#Create a dictionary of integer variables Slack that will be used to penalize solutions that select more than
#five edges for any node. Each Slack variable will have a value of 1 if the corresponding edge (i, j) is selected,
#and a value of 0 otherwise.
Slack = {}
for (i, j) in X.keys():
    Slack[(i, j)] = model.NewIntVar(0, 1, 'Slack_{}_{}'.format(i, j))
#Set the objective function to minimize the sum of the Slack variables
model.Minimize(sum(Slack[(i, j)] for (i, j) in X.keys()))
#Add constraints to ensure that each node has degree five, except for the last node, which has degree six
for k in NODES:
    model.Add(sum(X[(i, j)] for (i, j) in X.keys() if k in (i, j)) == degree[k])
#Add constraints to enforce that at least one edge is selected for each triple (i, j, k), where i < j < k
#and k is not equal to i or j
for (i, j) in X.keys():
    model.Add(X[(i, j)] + sum(Y[(i, j, k)] for k in NODES if k not in (i, j)) + Slack[(i, j)] >= 1)
#Add constraints to enforce that if Y[(i, j, k)] is selected, then either X[(i, k)] or X[(k, i)] must be selected,
#and either X[(j, k)] or X[(k, j)] must be selected
for (i, j) in X.keys():
    for k in NODES:
        if k not in (i, j):
            model.Add(Y[(i, j, k)] <= (X[(i, k)] if (i, k) in X.keys() else X[(k, i)]))
            model.Add(Y[(i, j, k)] <= (X[(j, k)] if (j, k) in X.keys() else X[(k, j)]))
#Create a new CpSolver object
solver = cp_model.CpSolver()
#Set the log_search_progress parameter to False to disable logging of the search progress
solver.parameters.log_search_progress = False
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    #X_sol = {key: solver.Value(X[key]) for key in X.keys()}
    #Y_sol = {key: solver.Value(Y[key]) for key in Y.keys()}
    EDGES = {(i, j) for (i, j) in X.keys() if solver.Value(X[(i, j)]) > 0.5}

    #print("X = ", X_sol)
    #print("Y = ", Y_sol)
    print("EDGES = ", EDGES)


EDGES =  {(6, 18), (12, 16), (4, 6), (4, 12), (3, 10), (3, 16), (12, 13), (4, 15), (14, 19), (5, 19), (8, 12), (2, 11), (2, 14), (1, 18), (1, 15), (1, 21), (13, 17), (13, 20), (16, 19), (7, 13), (15, 20), (18, 19), (4, 14), (3, 6), (5, 9), (14, 21), (5, 15), (9, 13), (2, 7), (8, 17), (2, 10), (1, 8), (19, 20), (10, 20), (1, 17), (7, 12), (6, 10), (16, 21), (7, 18), (3, 11), (20, 21), (4, 16), (5, 11), (3, 17), (5, 8), (14, 17), (8, 10), (9, 21), (9, 18), (11, 15), (2, 21), (7, 11), (6, 9)}
